In [ ]:
%reload_ext autoreload
%autoreload 2
import tie.dbutils as db
import pandas as pd
from scipy.io import savemat, loadmat
import json
from tqdm import tqdm
import os
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rc
import numpy as np
import matplotlib
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from statsmodels.stats.multitest import fdrcorrection
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as patches

from tqdm import notebook

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score, LeaveOneOut, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

import itertools
from sklearn.neighbors import KernelDensity
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

np.random.seed(42)
import shap

from xgboost import cv
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import matplotlib.colors as colors
import copy
import matplotlib.gridspec as gridspec

In [ ]:
rc('font', **{'family':'sans-serif',
              'sans-serif':['Helvetica']})
FONT_SIZE = 25
params = {'axes.labelsize': FONT_SIZE,
          'axes.titlesize': FONT_SIZE, 
          'legend.fontsize': FONT_SIZE, 
          'xtick.labelsize': FONT_SIZE, 
          'ytick.labelsize': FONT_SIZE}
matplotlib.rcParams.update(params)
# sns.set_theme(style="ticks", palette='twilight_shifted')

In [ ]:
info = loadmat('info_age_length_age_ml_v15.mat')
info.keys()
ages = info['ages']
genders = info['genders']
n_days = info['n_days']

ages = ages[n_days >= 7]
genders = genders[n_days >= 7]
n_days = n_days[n_days >= 7]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5))

## AGE
ax[0].hist(np.squeeze(ages), bins=20, color='grey')

ax[0].set_xlabel("Age [years]")
ax[0].set_ylabel("No. of subjects [count]")
ax[0].grid(False)

## DAYS
n, bins, _ = ax[1].hist(np.squeeze(n_days), 100, density=True, histtype='step',
                           cumulative=True, color='grey', alpha=1.0, linewidth=2)
# ax[1].step(np.cumsum(sorted(full_info["days"])[::-1]), np.cumsum(sorted(full_info["days"])[::-1]) / full_info["days"].sum(), color='grey')
# full_info["days"].hist(bins=20, ax=ax[1], color='grey')

ax[1].set_xlabel("Duration of smartphone recordings [days]")
ax[1].set_ylabel("Cumulative density function")
ax[1].grid(False)

ax[0].spines['right'].set_visible(False)
ax[0].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)

plt.tight_layout()
plt.savefig('figures/xgboost/figure_1_panel_b.pdf', transparent=True)

In [ ]:
df_fig1panb = pd.DataFrame(columns={'type', 'N', 'min', 'max', '25th', '50th', '75th'})
df_fig1panb['type'] = ['female', 'age', 'days']
df_fig1panb['N'] = [np.sum(genders == 2), len(ages), len(n_days)]
df_fig1panb['min'] = [np.nan, min(ages), min(n_days)]
df_fig1panb['max'] = [np.nan, max(ages), max(n_days)]
df_fig1panb['25th'] = [np.nan, np.percentile(ages, 25), np.percentile(n_days, 25)]
df_fig1panb['50th'] = [np.nan, np.percentile(ages, 50), np.percentile(n_days, 50)]
df_fig1panb['75th'] = [np.nan, np.percentile(ages, 75), np.percentile(n_days, 75)]
df_fig1panb.to_csv('./figures/xgboost/figure_1_panel_b.csv', index=False)
df_fig1panb

In [ ]:
days_dict = {'180': 0, '90': 1, '60': 2, '30': 3, '10': 4, '5': 5}
new_names = {i:f"{i // 50}-{i % 50}" for i in range(2500)}
new_names[2500] = "gender"
new_names[2501] = "log10(daily count)"
new_names[2502] = "entropy"
new_names[2503] = "screen-size"

In [ ]:
Xy_sp = loadmat('ml_age_data_special_full_v16')
X_sp = np.squeeze(Xy_sp['X_all'])
y_sp = np.squeeze(Xy_sp['y_all'])
g_sp = np.squeeze(Xy_sp['g_all'])
u_sp = np.squeeze(Xy_sp['u_all'])
e_sp = np.squeeze(Xy_sp['e_all'])
d_sp = np.squeeze(Xy_sp['d_all'])
n_sp = np.squeeze(Xy_sp['n_all'])
s_sp = np.squeeze(Xy_sp['s_all'])
print((X_sp.shape, y_sp.shape, g_sp.shape, u_sp.shape, e_sp.shape, d_sp.shape, n_sp.shape, s_sp.shape))

In [ ]:
D = days_dict['180']
X_180 = np.squeeze(np.array([a[D][0] for a in X_sp if a[D][0].shape[0] > 0]))
y_180 = np.squeeze(np.array([a[D][0][0] for a in y_sp if len(a[D][0][0]) > 0]))
g_180 = np.array([a[D][0][0] for a in g_sp if len(a[D][0][0]) > 0])
u_180 = np.log10(np.array([a[D][0][0] for a in u_sp if len(a[D][0][0]) > 0]) + 1e-15)
e_180 = np.array([a[D][0][0] for a in e_sp if len(a[D][0][0]) > 0])
d_180 = np.array([a[D][0][0] for a in d_sp if len(a[D][0][0]) > 0])
s_180 = np.array([a[D][0][0] for a in s_sp if len(a[D][0][0]) > 0])
X_180.shape, y_180.shape, g_180.shape, s_180.shape

In [ ]:
X_180 = np.concatenate([X_180, g_180, u_180, e_180, s_180], axis=1)
X_180.shape, y_180.shape

In [ ]:
X_df_180 = pd.DataFrame(X_180)
X_df_180 = X_df_180.rename(columns=new_names)
X_df_180.head()

In [ ]:
D = days_dict['90']
X_90 = np.squeeze(np.array([a[D][0] for a in X_sp if a[D][0].shape[0] > 0]))
y_90 = np.squeeze(np.array([a[D][0][0] for a in y_sp if len(a[D][0][0]) > 0]))
g_90 = np.array([a[D][0][0] for a in g_sp if len(a[D][0][0]) > 0])
u_90 = np.log10(np.array([a[D][0][0] for a in u_sp if len(a[D][0][0]) > 0]) + 1e-15)
e_90 = np.array([a[D][0][0] for a in e_sp if len(a[D][0][0]) > 0])
d_90 = np.array([a[D][0][0] for a in d_sp if len(a[D][0][0]) > 0])
s_90 = np.array([a[D][0][0] for a in s_sp if len(a[D][0][0]) > 0])
X_90.shape, y_90.shape, g_90.shape, s_90.shape

X_90 = np.concatenate([X_90, g_90, u_90, e_90, s_90], axis=1)
X_90.shape, y_180.shape

X_df_90 = pd.DataFrame(X_90)
X_df_90 = X_df_90.rename(columns=new_names)
X_df_90.head()

In [ ]:
data_dmatrix = xgb.DMatrix(data=X_df_180,label=y_180)

params = {'objective':'reg:squarederror',  'colsample_bytree': 0.6,'learning_rate': 0.01, 
          'subsample':0.7, 'max_depth': 5, 'reg_alpha': 0.001, 'reg_lambda': 0, 'min_child_weight': 8}

# Single model

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
TH = 7
all_train_pred = []
all_train_true = []
all_test_pred = []
all_test_true = []
all_models = []
train_used = 0

for train_index, test_index in kf.split(X_df_180):
#     print("TRAIN:", train_index[:3], "TEST:", test_index[:3])
    X_train, X_test = X_df_180.iloc[train_index], X_df_180.iloc[test_index]
    y_train, y_test = y_180[train_index], y_180[test_index]
#     print(X_train.shape, X_test.shape)
    d_train, d_test = np.squeeze(d_180[train_index]), np.squeeze(d_180[test_index])
    
    X_train = X_train[d_train >= TH]
    y_train = y_train[d_train >= TH]
    train_used += np.sum(d_train >= TH)
    print(f"TR {np.sum(d_train >= TH) / len(d_train) * 100:.2f}%")
    
    model = xgb.XGBRegressor(n_estimators=560, **params).fit(X_train, y_train)
    all_models.append(model)
    pred_vals_train = model.predict(X_train)
    pred_vals_test = model.predict(X_test)
    
    all_train_pred.extend(pred_vals_train)
    all_test_pred.extend(pred_vals_test)
    all_train_true.extend(y_train)
    all_test_true.extend(y_test)
    print(f"TR : {np.mean(np.abs(np.array(all_train_pred) - np.array(all_train_true))):.2f} - TE : {np.mean(np.abs(np.array(all_test_pred) - np.array(all_test_true))):.2f}")

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
TH = 7
all_train_pred_90 = []
all_train_true_90 = []
all_test_pred_90 = []
all_test_true_90 = []
all_models_90 = []
train_used = 0

for train_index, test_index in kf.split(X_df_90):
#     print("TRAIN:", train_index[:3], "TEST:", test_index[:3])
    X_train, X_test = X_df_90.iloc[train_index], X_df_90.iloc[test_index]
    y_train, y_test = y_90[train_index], y_90[test_index]
#     print(X_train.shape, X_test.shape)
    d_train, d_test = np.squeeze(d_90[train_index]), np.squeeze(d_90[test_index])
    
    X_train = X_train[d_train >= TH]
    y_train = y_train[d_train >= TH]
    train_used += np.sum(d_train >= TH)
    print(f"TR {np.sum(d_train >= TH) / len(d_train) * 100:.2f}%")
    
    model = xgb.XGBRegressor(n_estimators=560, **params).fit(X_train, y_train)
    all_models_90.append(model)
    pred_vals_train = model.predict(X_train)
    pred_vals_test = model.predict(X_test)
    
    all_train_pred_90.extend(pred_vals_train)
    all_test_pred_90.extend(pred_vals_test)
    all_train_true_90.extend(y_train)
    all_test_true_90.extend(y_test)
    print(f"TR : {np.mean(np.abs(np.array(all_train_pred_90) - np.array(all_train_true_90))):.2f} - TE : {np.mean(np.abs(np.array(all_test_pred_90) - np.array(all_test_true_90))):.2f}")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4.6))

pred_vals = np.array(all_test_pred)
real_vals = np.array(all_test_true)

s_idx = np.argsort(real_vals)
ax.scatter(np.arange(len(s_idx)),real_vals[s_idx], marker='.', c='b', label='Chronological age')  
ax.scatter(np.arange(len(s_idx)), pred_vals[s_idx], marker='.', s=100, c='orange', label='Predicted age')  
ax.grid(False)
ax.legend(loc='upper center',  ncol=1, bbox_to_anchor=[.5, 1.3])

ax.set_ylabel('Age [years]')
ax.set_xlabel('Subject No. (Sorted by age)');
ax.set_ylim([0, 100])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.tight_layout();
plt.savefig('figures/xgboost/figure_1_panel_c.pdf')

## Explain

In [ ]:
all_shap = []
all_test = []
for i, (train_index, test_index) in enumerate(kf.split(X_df_180)):
    _, X_test = X_df_180.iloc[train_index], X_df_180.iloc[test_index]
    _, y_test = y_180[train_index], y_180[test_index]
    explainer = shap.Explainer(all_models[i])
    shap_values = explainer(X_test)
    all_shap.extend(shap_values)
    all_test.extend(y_test)


In [ ]:
fig = plt.figure(figsize=(20, 8))
gs0 = gridspec.GridSpec(1, 4, figure=fig, width_ratios=[1, 0.05, 1, 0.05])

labels = ['0.03', '0.1', '1', '4', '20']
ticks = np.arange(50)[::10]

shap_values = all_shap
pos_val = np.zeros((len(shap_values), 50, 50)) * np.nan
neg_val = np.zeros((len(shap_values), 50, 50)) * np.nan
pos_extra = np.zeros((len(shap_values), 4)) * np.nan
neg_extra = np.zeros((len(shap_values), 4)) * np.nan
for IDX in range(len(shap_values)):
    _pos = copy.deepcopy(np.reshape(shap_values[IDX].values[:2500], (50, 50)))
    _neg = copy.deepcopy(np.reshape(shap_values[IDX].values[:2500], (50, 50)))
    _pos_e = copy.deepcopy(shap_values[IDX].values[2500:])
    _neg_e = copy.deepcopy(shap_values[IDX].values[2500:])
    
    _pos[_pos < 0] = 0
    pos_val[IDX] = _pos
    _pos_e[_pos_e < 0] = 0
    pos_extra[IDX] = _pos_e
    
    _neg[_neg > 0] = 0
    neg_val[IDX] = _neg
    _neg_e[_neg_e > 0] = 0
    neg_extra[IDX] = _neg_e

    
m_pos = np.nanmean(pos_val, 0)
mask = np.ones_like(m_pos)
mask[m_pos < 0.01] = np.nan
ax0 = fig.add_subplot(gs0[0, 0])
im = ax0.imshow(m_pos * mask, cmap='Reds', vmax=1.2, vmin=0, aspect='auto')
ax0.invert_yaxis()
divider = make_axes_locatable(ax0)
cax = divider.new_vertical(size="5%", pad=.1, pack_start=False)
fig.add_axes(cax)
cb = fig.colorbar(im, cax=cax, orientation="horizontal")
cb.ax.xaxis.set_ticks_position('top')
ax0.set_xticks(ticks)
ax0.set_xticklabels(labels)
ax0.xaxis.set_tick_params(rotation=45)
ax0.set_yticks(ticks)
ax0.set_yticklabels(labels)
ax0.set_ylabel(r'$ITI(k + 1)$ [s]')
ax0.set_xlabel(r'$ITI(k)$ [s]')
ax0.text(-11, 50.1, 'Mean\nShapley\nvalue', size=25)
    
    
ax01 = fig.add_subplot(gs0[0, 1])
ax01.imshow(np.nanmean(pos_extra, 0)[:, None], vmax=1.2, vmin=0, cmap='Reds', aspect='auto')
ax01.yaxis.tick_right()
ax01.set_xticks([])
ax01.set_yticks([0, 1, 2, 3])
ax01.set_yticklabels([new_names[2500], new_names[2501], new_names[2502], new_names[2503]], rotation=90, size=15, verticalalignment='center')

# ax99 = fig.add_subplot(gs0[0, 2])
# ax99.set_xticks([])
# ax99.set_yticks([])
# ax99.spines['right'].set_visible(False)
# ax99.spines['top'].set_visible(False)
# ax99.spines['left'].set_visible(False)
# ax99.spines['bottom'].set_visible(False)

m_neg = np.nanmean(neg_val, 0)
mask = np.ones_like(m_neg)
mask[m_neg > -0.01] = np.nan
ax1 = fig.add_subplot(gs0[0, 2])
im = ax1.imshow(m_neg * mask, cmap="Blues_r", vmin=-0.8, vmax=0, aspect='auto')
divider = make_axes_locatable(ax1)
cax = divider.new_vertical(size="5%", pad=.1, pack_start=False)
fig.add_axes(cax)
cb = fig.colorbar(im, cax=cax, orientation="horizontal")
cb.ax.xaxis.set_ticks_position('top')
ax1.invert_yaxis()
ax1.set_xticks(ticks)
ax1.set_xticklabels(labels)
ax1.xaxis.set_tick_params(rotation=45)
ax1.set_yticks(ticks)
ax1.set_yticklabels(labels)
ax1.set_ylabel(r'$ITI(k + 1)$ [s]')
ax1.set_xlabel(r'$ITI(k)$ [s]')
ax1.text(-11, 50.1, 'Mean\nShapley\nvalue', size=25)

ax11 = fig.add_subplot(gs0[0, 3])
ax11.imshow(np.nanmean(neg_extra, 0)[:, None], vmin=-0.8, vmax=0, cmap='Blues_r', aspect='auto')
ax11.yaxis.tick_right()
ax11.set_xticks([])
ax11.set_yticks([0, 1, 2, 3])
ax11.set_yticklabels([new_names[2500], new_names[2501], new_names[2502], new_names[2503]], rotation=90, size=15, verticalalignment='center')

plt.tight_layout()
plt.savefig("figures/xgboost/figure_1_panel_e.pdf")

## Time test

In [ ]:
kf = KFold(n_splits=10, shuffle=True)

all_test_pred = []
all_test_true = []
all_regr_sp = []
all_train_pred = []
all_train_true = []

for train_index, test_index in kf.split(X_sp):
    X_train = X_sp[train_index]
    y_train = y_sp[train_index]
    X_test = X_sp[test_index]
    y_test = y_sp[test_index]
    g_train = g_sp[train_index]
    g_test = g_sp[test_index]
    u_train = u_sp[train_index]
    u_test = u_sp[test_index]
    e_train = e_sp[train_index]
    e_test = e_sp[test_index]
    d_train = d_sp[train_index]
    d_test = d_sp[test_index]
    s_train = s_sp[train_index]
    s_test = s_sp[test_index]
    
    D = days_dict['180']
    X_d = np.squeeze(np.array([a[D][0] for a in X_train if a[D][0].shape[0] > 0]))
    y_tr = np.squeeze(np.array([a[D][0][0] for a in y_train if len(a[D][0][0]) > 0]))
    g_d = np.array([a[D][0][0] for a in g_train if len(a[D][0][0]) > 0])
    u_d = np.log10(np.array([a[D][0][0] for a in u_train if len(a[D][0][0]) > 0]) + 1e-15)
    e_d = np.array([a[D][0][0] for a in e_train if len(a[D][0][0]) > 0])
    d_d = np.squeeze(np.array([a[D][0][0] for a in d_train if len(a[D][0][0]) > 0]))
    s_d = np.array([a[D][0][0] for a in s_train if len(a[D][0][0]) > 0])
    X_d = np.concatenate([X_d, g_d, u_d, e_d, s_d], axis=1)
    
    X_d = X_d[d_d >= TH]
    y_tr = y_tr[d_d >= TH]
    train_used += np.sum(d_d >= TH)
    print(f"TR {np.sum(d_d >= TH) / len(d_d) * 100:.2f}%")
    
    X_df_train = pd.DataFrame(X_d)
    X_df_train = X_df_train.rename(columns=new_names)
    
    X_d = np.squeeze(np.array([a[D][0] for a in X_test if a[D][0].shape[0] > 0]))
    y_te = np.squeeze(np.array([a[D][0][0] for a in y_test if len(a[D][0][0]) > 0]))
    g_d = np.array([a[D][0][0] for a in g_test if len(a[D][0][0]) > 0])
    u_d = np.log10(np.array([a[D][0][0] for a in u_test if len(a[D][0][0]) > 0]) + 1e-15)
    e_d = np.array([a[D][0][0] for a in e_test if len(a[D][0][0]) > 0])
    d_d = np.array([a[D][0][0] for a in d_test if len(a[D][0][0]) > 0])
    s_d = np.array([a[D][0][0] for a in s_test if len(a[D][0][0]) > 0])
    X_d = np.concatenate([X_d, g_d, u_d, e_d, s_d], axis=1)
    X_df_test = pd.DataFrame(X_d)
    X_df_test = X_df_test.rename(columns=new_names)

    model = xgb.XGBRegressor(n_estimators=559, tree_method='gpu_hist', **params).fit(X_df_train, y_tr)
    all_regr_sp.append(model)
    pred_vals_train = model.predict(X_df_train)
    pred_vals_test = model.predict(X_df_test)
    
    all_train_pred.extend(pred_vals_train)
    all_train_true.extend(y_tr)
    
    to_append_pred_test = [pred_vals_test]
    to_append_true_test = [y_te]
    
    for _days in ['90', '60', '30', '10', '5']:
        D = days_dict[_days]
        X_d = np.squeeze(np.array([a[D][0] for a in X_test if a[D][0].shape[0] > 0]))
        y_d = np.squeeze(np.array([a[D][0][0] for a in y_test if len(a[D][0][0]) > 0]))
        g_d = np.array([a[D][0][0] for a in g_test if len(a[D][0][0]) > 0])
        u_d = np.log10(np.array([a[D][0][0] for a in u_test if len(a[D][0][0]) > 0]) + 1e-15)
        e_d = np.array([a[D][0][0] for a in e_test if len(a[D][0][0]) > 0])
        d_d = np.array([a[D][0][0] for a in d_test if len(a[D][0][0]) > 0])
        s_d = np.array([a[D][0][0] for a in s_test if len(a[D][0][0]) > 0])
        X_d = np.concatenate([X_d, g_d, u_d, e_d, s_d], axis=1)
        X_df_d = pd.DataFrame(X_d)
        X_df_d = X_df_d.rename(columns=new_names)
        pred_vals_d = model.predict(X_df_d)
        
        to_append_pred_test.append(pred_vals_d)
        to_append_true_test.append(y_d)
        
    all_test_pred.append(to_append_pred_test)
    all_test_true.append(to_append_true_test)

In [ ]:
errors_mae = [[] for _ in range(6)]
errors = [[] for _ in range(6)]
all_r2_pred = [[] for _ in range(6)]
all_r2_true = [[] for _ in range(6)]

for days in range(6):
    for fold in range(10):
        errors_mae[days].extend(np.abs(all_test_pred[fold][days] - all_test_true[fold][days]))
        errors[days].extend(all_test_pred[fold][days] - all_test_true[fold][days])
        all_r2_pred[days].extend(all_test_pred[fold][days])
        all_r2_true[days].extend(all_test_true[fold][days])

In [ ]:
a = pd.concat([pd.DataFrame({"error": errors[i], "error_mae": errors_mae[i], "days": np.ones(len(errors[i])) * d}) for i, d in enumerate([180, 90, 60, 30, 10, 5])], 0)

fig, ax = plt.subplots(1, 1, figsize=(6, 5))
sns.lineplot(data=a.reset_index(drop=True), x='days', y='error_mae', ax=ax, marker='p', markersize=10)
ax.grid(False)
ax.set_xlabel("Duration of recording\nconsidered [Max. days]")
ax.set_ylabel("MAE [years]")
ax.set(xscale="log")
ax.set_xticks([5, 10, 30, 60, 90, 180]);
ax.get_xaxis().get_major_formatter().labelOnlyBase = False
ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.tight_layout()
plt.savefig('figures/xgboost/figure_1_panel_d.pdf')

df_fig1pand = pd.DataFrame(columns={'N', '25th', '50th', '75th', 'mean', 'MAE', 'R2', 'days'})
df_fig1pand['days'] = ['5', '10', '30', '60', '90', '180']
df_fig1pand['N'] = [len(a[a['days'] == 5]['error']), 
                    len(a[a['days'] == 10]['error']), 
                    len(a[a['days'] == 30]['error']),
                    len(a[a['days'] == 60]['error']), 
                    len(a[a['days'] == 90]['error']), 
                    len(a[a['days'] == 180]['error'])]

df_fig1pand['25th'] = [np.percentile(a[a['days'] == 5]['error'], 25), 
                       np.percentile(a[a['days'] == 10]['error'], 25), 
                       np.percentile(a[a['days'] == 30]['error'], 25), 
                       np.percentile(a[a['days'] == 60]['error'], 25), 
                       np.percentile(a[a['days'] == 90]['error'], 25), 
                       np.percentile(a[a['days'] == 180]['error'], 25)]

df_fig1pand['50th'] = [np.percentile(a[a['days'] == 5]['error'], 50), 
                       np.percentile(a[a['days'] == 10]['error'], 50), 
                       np.percentile(a[a['days'] == 30]['error'], 50), 
                       np.percentile(a[a['days'] == 60]['error'], 50), 
                       np.percentile(a[a['days'] == 90]['error'], 50), 
                       np.percentile(a[a['days'] == 180]['error'], 50)]

df_fig1pand['75th'] = [np.percentile(a[a['days'] == 5]['error'], 75), 
                       np.percentile(a[a['days'] == 10]['error'], 75), 
                       np.percentile(a[a['days'] == 30]['error'], 75), 
                       np.percentile(a[a['days'] == 60]['error'], 75), 
                       np.percentile(a[a['days'] == 90]['error'], 75), 
                       np.percentile(a[a['days'] == 180]['error'], 75)]

df_fig1pand['mean'] = [np.mean(a[a['days'] == 5]['error']), 
                       np.mean(a[a['days'] == 10]['error']), 
                       np.mean(a[a['days'] == 30]['error']), 
                       np.mean(a[a['days'] == 60]['error']), 
                       np.mean(a[a['days'] == 90]['error']), 
                       np.mean(a[a['days'] == 180]['error'])]

df_fig1pand['MAE'] = [np.mean(a[a['days'] == 5]['error'].abs()), 
                      np.mean(a[a['days'] == 10]['error'].abs()), 
                      np.mean(a[a['days'] == 30]['error'].abs()), 
                      np.mean(a[a['days'] == 60]['error'].abs()), 
                      np.mean(a[a['days'] == 90]['error'].abs()), 
                      np.mean(a[a['days'] == 180]['error'].abs())]

df_fig1pand['R2'] = [r2_score(all_r2_true[5], all_r2_pred[5]), 
                      r2_score(all_r2_true[4], all_r2_pred[4]),
                      r2_score(all_r2_true[3], all_r2_pred[3]),
                      r2_score(all_r2_true[2], all_r2_pred[2]),
                      r2_score(all_r2_true[1], all_r2_pred[1]),
                      r2_score(all_r2_true[0], all_r2_pred[0]),
                     ]

df_fig1pand.to_csv('figures/xgboost/figure_1_panel_d.csv', index=False)
df_fig1pand

# Stroke

In [ ]:
Xy_stroke_all = loadmat('ml_age_STROKE_data_v14.mat')
X_stroke_all = np.squeeze(Xy_stroke_all['X_all'])
y_stroke_all = np.squeeze(Xy_stroke_all['y_all'])
g_stroke_all = np.squeeze(Xy_stroke_all['g_all'])
u_stroke_all = np.squeeze(Xy_stroke_all['u_all'])
e_stroke_all = np.squeeze(Xy_stroke_all['e_all'])
s_stroke_all = np.squeeze(Xy_stroke_all['s_all'])
print((X_stroke_all.shape, y_stroke_all.shape, g_stroke_all.shape, u_stroke_all.shape, e_stroke_all.shape, s_stroke_all.shape))


In [ ]:
X_stroke_test = X_stroke_all[:]
y_stroke_test = y_stroke_all[:]
g_stroke_test = g_stroke_all[:]
u_stroke_test = u_stroke_all[:]
e_stroke_test = e_stroke_all[:]
s_stroke_test = s_stroke_all[:]

# only first
X_stroke_sub_test = np.array([a[:, 0] for a in X_stroke_test])
y_stroke_sub_test = np.squeeze(np.array([a[0] for a in y_stroke_test]))
g_stroke_sub_test = np.array([a[0] for a in g_stroke_test]) * 1
u_stroke_sub_test = np.log10(np.array([a[0] for a in u_stroke_test]) + 1e-15) * 1
e_stroke_sub_test = np.array([a[0] for a in e_stroke_test]) * 1
s_stroke_sub_test = np.array([a[0] for a in s_stroke_test])


# put tog.
X_stroke_test = np.concatenate([X_stroke_sub_test[:, :], g_stroke_sub_test, u_stroke_sub_test, e_stroke_sub_test, s_stroke_sub_test], -1)
print(X_stroke_test.shape)

In [ ]:
X_s_df = pd.DataFrame(X_stroke_test)
X_s_df = X_s_df.rename(columns=new_names)
X_s_df.head()

In [ ]:
all_pred_stroke_ages = []

for _model in all_models_90:
    pred_vals_test_s = _model.predict(X_s_df)
    all_pred_stroke_ages.append(pred_vals_test_s)

In [ ]:
all_pred_zeros = [np.int32(a.predict(X_s_df * 0)[0]) for a in all_models_90]
print(all_pred_zeros)

In [ ]:
stacked_stroke = np.vstack(all_pred_stroke_ages)
min_s_stroke = np.percentile(stacked_stroke, 2.5, 0)
max_s_stroke = np.percentile(stacked_stroke, 97.5, 0)
med_s_stroke = np.percentile(stacked_stroke, 50, 0)

In [ ]:
error_90 = np.array(all_test_true_90) - np.array(all_test_pred_90)
chained_real = np.array(all_test_true_90)
chained_90 = np.array(all_test_pred_90)
all_real_90 = np.array(all_test_true_90)
all_pred_90 = np.array(all_test_pred_90)

In [ ]:
# subsample based on stroke distribution
nboot = 10000
subsets_true = []
subsets_pred = []
for _ in range(nboot):
    to_add_true = []
    to_add_pred = []
    for x in y_stroke_sub_test:
        _true_matched = chained_real[chained_real == x]
        _pred_matched = chained_90[chained_real == x]
        if len(_true_matched) == 0:  # if I don't have 83 I try with 84 anyway is close enough to match the distro
            _true_matched = chained_real[chained_real == x + 1]
            _pred_matched = error_90[chained_real == x + 1]
        _idx = np.random.choice(len(_true_matched))
        to_add_true.append(_true_matched[_idx])
        to_add_pred.append(_pred_matched[_idx])
    subsets_pred.append(np.array(to_add_pred))
    subsets_true.append(np.array(to_add_true))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(22, 10))

idx = np.argsort(y_stroke_sub_test)

ss = np.linspace(-0.0, 40, 41)

ax[1].fill_between(ss, min_s_stroke[idx], max_s_stroke[idx], color='k', alpha=0.1, zorder=3)
ax[1].scatter(np.arange(len(idx)), y_stroke_sub_test[idx], marker='o', color='blue', label='Chronological age')
ax[1].scatter(np.arange(len(idx)), med_s_stroke[idx], marker='o', c='orange', label='Predicted age')
ax[1].set_xlabel('Subject No. (Sorted by age)')
ax[1].set_ylabel('Age [years]')
# ax[1].plot([0, 40], [np.mean(ages), np.mean(ages)], 'k--', alpha=1.0, label='Mean age (healthy training set)')
ax[1].plot([0, 40], [np.mean(all_pred_zeros), np.mean(all_pred_zeros)], 'r--', alpha=1.0, label='Default output')
ax[1].fill_between(ss, np.percentile(all_pred_zeros, 2.5), np.percentile(all_pred_zeros, 97.5), color='r', alpha=0.1, zorder=3)

handles, labels = ax[1].get_legend_handles_labels()
order = [1, 2, 0]
ax[1].legend([handles[idx] for idx in order],[labels[idx] for idx in order], loc='upper left', shadow=False, fancybox=False, ncol=1, bbox_to_anchor=[0.0, 1.6])

ax[1].grid(False)
ax[1].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)
ax[1].set_ylim([18, 85])
X = np.array([np.median(a - y_stroke_sub_test) for a in all_pred_stroke_ages])
# X = list(itertools.chain.from_iterable([a - y_stroke_sub_test for a in all_pred_stroke_ages]))
# ax[0].hist(X)
sns.kdeplot(X, ax=ax[0], bw_method=.6, linewidth=3, label='Stroke survivors', color='blue')
Y = [np.median(a - b) for a, b in zip(subsets_true, subsets_pred)]
# Y = list(itertools.chain.from_iterable([a - b for a, b in zip(subsets_true, subsets_pred)]))
sns.kdeplot(Y, ax=ax[0], bw_method=.3, linewidth=3, label='Healthy population (age matched)', color='g')
# ax[0].hist(Y, color='g')

ax[0].spines['right'].set_visible(False)
ax[0].spines['top'].set_visible(False)
ax[0].legend(shadow=False, fancybox=False, ncol=1, bbox_to_anchor=[.7, 1.6])
ax[0].set_xlabel("Error [years]")
ax[0].set_ylabel("Probability density")
ax[0].grid(False)
ax[0].set_xlim([-10, 13])
ax[0].set_ylim([0, 0.6])

plt.tight_layout()

plt.savefig('./figures/xgboost/figure_2_panel_d_stroke.pdf')

In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(X, Y)

df_fig2_stroke = pd.DataFrame({'type': ['stroke distro', 'healthy distro', 't-test'], 
                        'p-val': [np.nan, np.nan, pval],
                        't-val': [np.nan, np.nan, tval], 
                        '25th': [np.percentile(X, 25),np.percentile(Y, 25), np.nan],
                        '50th': [np.percentile(X, 50),np.percentile(Y, 50), np.nan],
                        '75th': [np.percentile(X, 75),np.percentile(Y, 75), np.nan],
                        'mean': [np.mean(X),np.mean(Y), np.nan],
                        'MAE': [np.mean(np.abs(y_stroke_sub_test - med_s_stroke)),np.nan, np.nan]
                           })
df_fig2_stroke.to_csv('./figures/xgboost/fig2_stroke_stats.csv')
df_fig2_stroke

In [ ]:
df_ = pd.DataFrame({'Age': y_stroke_sub_test[idx], 'Pred' : med_s_stroke[idx]})
fig, ax = plt.subplots(1,1, figsize=(5, 5))
sns.regplot(y="Pred", x="Age", data=df_);
ax.set_xlabel('Age')
ax.set_ylabel('Pred');
plt.tight_layout()
plt.grid(False)
plt.savefig('./figures/xgboost/figure_2_insert_stroke.pdf')
savemat('figure_2_xgboost_insert_stroke.mat', {'real_str': y_stroke_sub_test[idx], 'pred_str': med_s_stroke[idx]})

# Epilepsy

In [ ]:
Xy_rns_all = loadmat('ml_age_RNS_data_v15.mat')
X_rns_all = np.squeeze(Xy_rns_all['X_all'])
y_rns_all = np.squeeze(Xy_rns_all['y_all'])
g_rns_all = np.squeeze(Xy_rns_all['g_all'])
u_rns_all = np.squeeze(Xy_rns_all['u_all'])
e_rns_all = np.squeeze(Xy_rns_all['e_all'])
s_rns_all = np.squeeze(Xy_rns_all['s_all'])
partId_rns_all = np.squeeze(Xy_rns_all['partId_all'])
print((X_rns_all.shape, y_rns_all.shape, g_rns_all.shape, u_rns_all.shape, e_rns_all.shape, s_rns_all.shape, partId_rns_all.shape))

In [ ]:
X_rns_test = X_rns_all[:]
y_rns_test = y_rns_all[:]
g_rns_test = g_rns_all[:]
u_rns_test = u_rns_all[:]
e_rns_test = e_rns_all[:]
s_rns_test = s_rns_all[:]

# only first
X_rns_sub_test = np.array([a[:, 0] for a in X_rns_test])
y_rns_sub_test = np.squeeze(np.array([a[0] for a in y_rns_test]))
g_rns_sub_test = np.array([a[0] for a in g_rns_test]) * 1
u_rns_sub_test = np.log10(np.array([a[0] for a in u_rns_test]) + 1e-15) * 1
e_rns_sub_test = np.array([a[0] for a in e_rns_test]) * 1
s_rns_sub_test = np.array([a[0] for a in s_rns_test]) * 1


# put tog.
X_rns_test = np.concatenate([X_rns_sub_test[:, :], g_rns_sub_test, u_rns_sub_test, e_rns_sub_test, s_rns_sub_test], -1)
print(X_rns_test.shape)

In [ ]:
X_r_df = pd.DataFrame(X_rns_test)
X_r_df = X_r_df.rename(columns=new_names)
X_r_df.head()

In [ ]:
all_pred_rns_ages = [a.predict(X_r_df) for a in all_models]

In [ ]:
Xy_epi_all = loadmat('ml_age_epi_data_v16.mat')
X_epi_all = np.squeeze(Xy_epi_all['X_all'])
y_epi_all = np.squeeze(Xy_epi_all['y_all'])
g_epi_all = np.squeeze(Xy_epi_all['g_all'])
u_epi_all = np.squeeze(Xy_epi_all['u_all'])
e_epi_all = np.squeeze(Xy_epi_all['e_all'])
s_epi_all = np.squeeze(Xy_epi_all['s_all'])
partId_epi_all = np.squeeze(Xy_epi_all['partId_all'])
print((X_epi_all.shape, y_epi_all.shape, g_epi_all.shape, u_epi_all.shape, e_epi_all.shape, s_epi_all.shape, partId_epi_all.shape))


In [ ]:
X_epi_test = X_epi_all[:]
y_epi_test = y_epi_all[:]
g_epi_test = g_epi_all[:]
u_epi_test = u_epi_all[:]
e_epi_test = e_epi_all[:]
s_epi_test = s_epi_all[:]

# only first
X_epi_sub_test = np.array([a[:, 0] for a in X_epi_test])
y_epi_sub_test = np.squeeze(np.array([a[0] for a in y_epi_test]))
g_epi_sub_test = np.array([a[0] for a in g_epi_test]) * 1
u_epi_sub_test = np.log10(np.array([a[0] for a in u_epi_test]) + 1e-15) * 1
e_epi_sub_test = np.array([a[0] for a in e_epi_test]) * 1
s_epi_sub_test = np.array([a[0] for a in s_epi_test]) * 1


# put tog.
X_epi_test = np.concatenate([X_epi_sub_test[:, :], g_epi_sub_test, u_epi_sub_test, e_epi_sub_test, s_epi_sub_test], -1)
print(X_epi_test.shape)

In [ ]:
X_e_df = pd.DataFrame(X_epi_test)
X_e_df = X_e_df.rename(columns=new_names)
X_e_df.head()

In [ ]:
all_pred_epi_ages = [a.predict(X_e_df) for a in all_models]

In [ ]:
all_pred_rns_ages[3].shape

In [ ]:
all_pred_epilepsy_ages = np.concatenate([all_pred_rns_ages, all_pred_epi_ages], axis=1)
all_study_type = np.concatenate([np.ones_like(y_rns_sub_test), 2 * np.ones_like(y_epi_sub_test)])
y_epilepsy_sub_test = np.concatenate([y_rns_sub_test, y_epi_sub_test])
partId_epilepsy_sub_test = np.concatenate([partId_rns_all, partId_epi_all])
all_pred_epilepsy_ages.shape, y_epilepsy_sub_test.shape, partId_epilepsy_sub_test.shape

In [ ]:
len(all_test_true[0][0])

In [ ]:
error_all = np.array(all_test_true) - np.array(all_test_pred)
chained_real_all = np.array(all_test_true)
chained_all = np.array(all_test_pred)

In [ ]:
# subsample based on stroke distribution
nboot = 10000
subsets_true = []
subsets_pred = []
for _ in range(nboot):
    to_add_true = []
    to_add_pred = []
    for x in y_epilepsy_sub_test:
        _true_matched = chained_real_all[chained_real_all == x]
        _pred_matched = chained_all[chained_real_all == x]
        if len(_true_matched) == 0:  # if I don't have 83 I try with 84 anyway is close enough to match the distro
            _true_matched = chained_real_all[chained_real_all == x + 4]
            _pred_matched = error_all[chained_real_all == x + 4]
        _idx = np.random.choice(len(_true_matched))
        to_add_true.append(_true_matched[_idx])
        to_add_pred.append(_pred_matched[_idx])
    subsets_pred.append(np.array(to_add_pred))
    subsets_true.append(np.array(to_add_true))

In [ ]:
stacked_epilepsy = all_pred_epilepsy_ages
min_s = np.percentile(stacked_epilepsy, 2.5, 0)
max_s = np.percentile(stacked_epilepsy, 97.5, 0)
med_s = np.percentile(stacked_epilepsy, 50, 0)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(22, 10))
idx = np.argsort(y_epilepsy_sub_test)

ss = np.linspace(-0.0, len(idx) - 1, len(idx))

ax[1].scatter(np.arange(len(idx))[all_study_type[idx] == 1], y_epilepsy_sub_test[idx][all_study_type[idx] == 1], s=100, marker='o', c='blue', label='Chronological age')
ax[1].scatter(np.arange(len(idx))[all_study_type[idx] == 2], y_epilepsy_sub_test[idx][all_study_type[idx] == 2], s=40, marker='o', c='blue')
ax[1].fill_between(ss, min_s[idx], max_s[idx], color='k', alpha=0.1, zorder=3)
ax[1].scatter(np.arange(len(idx)), med_s[idx], marker='o', c='orange', label='Predicted age')

ax[1].set_xlabel('Subject No. (Sorted by age)')
ax[1].set_ylabel('Age [years]')
# ax[1].plot([0, 40], [np.mean(ages), np.mean(ages)], 'k--', alpha=1.0, label='Mean age (healthy training set)')
ax[1].plot([0, len(idx)], [np.mean(all_pred_zeros), np.mean(all_pred_zeros)], 'r--', alpha=1.0, label='Default output')
ax[1].fill_between(ss, np.percentile(all_pred_zeros, 2.5), np.percentile(all_pred_zeros, 97.5), color='r', alpha=0.1, zorder=3)

handles, labels = ax[1].get_legend_handles_labels()
order = [1, 2, 0]
ax[1].legend([handles[idx] for idx in order],[labels[idx] for idx in order], loc='upper left', shadow=False, fancybox=False, ncol=1, bbox_to_anchor=[0.0, 1.6])

ax[1].grid(False)
ax[1].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)
ax[1].set_ylim([18, 85])

X = np.array([np.median(a - y_epilepsy_sub_test) for a in all_pred_epilepsy_ages])
# X = list(itertools.chain.from_iterable([a - y_stroke_sub_test for a in all_pred_stroke_ages]))
# ax[0].hist(X)
sns.kdeplot(X, ax=ax[0], bw_method=.6, linewidth=3, label='Epilepsy', color='blue')
Y = [np.median(a - b) for a, b in zip(subsets_true, subsets_pred)]
# Y = list(itertools.chain.from_iterable([a - b for a, b in zip(subsets_true, subsets_pred)]))
sns.kdeplot(Y, ax=ax[0], bw_method=.3, linewidth=3, label='Healthy population (age matched)', color='g')
# ax[0].hist(Y, color='g')

ax[0].spines['right'].set_visible(False)
ax[0].spines['top'].set_visible(False)
ax[0].legend(shadow=False, fancybox=False, ncol=1, bbox_to_anchor=[.7, 1.6])
ax[0].set_xlabel("Error [years]")
ax[0].set_ylabel("Probability density")
ax[0].grid(False)
ax[0].set_xlim([-10, 13])
ax[0].set_ylim([0, 0.7])

plt.tight_layout()

plt.savefig('./figures/xgboost/figure_2_panel_d_epilepsy.pdf')

In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(X, Y)

df_fig2_epi = pd.DataFrame({'type': ['epi distro', 'healthy distro', 't-test'], 
                        'p-val': [np.nan, np.nan, pval],
                        't-val': [np.nan, np.nan, tval], 
                        '25th': [np.percentile(X, 25),np.percentile(Y, 25), np.nan],
                        '50th': [np.percentile(X, 50),np.percentile(Y, 50), np.nan],
                        '75th': [np.percentile(X, 75),np.percentile(Y, 75), np.nan],
                         'mean': [np.mean(X),np.mean(Y), np.nan],
                            'MAE': [np.mean(np.abs(y_epilepsy_sub_test - med_s)),np.nan, np.nan]
                           })
df_fig2_epi.to_csv('./figures/xgboost/fig2_epi_stats_2.csv')
df_fig2_epi



In [ ]:
df_ = pd.DataFrame({'Age': y_epilepsy_sub_test[idx], 'Pred' : med_s[idx]})
fig, ax = plt.subplots(1,1, figsize=(5, 5))
sns.regplot(y="Pred", x="Age", data=df_);
ax.set_xlabel('Age')
ax.set_ylabel('Pred');
plt.tight_layout()
plt.grid(False)
plt.savefig('./figures/xgboost/figure_2_insert_epilepsy.pdf')
savemat('figure_2_xgboost_insert_epi.mat', {'real_epi': y_epilepsy_sub_test[idx], 'pred_epi': med_s[idx]})

In [ ]:
len(idx)

# Explain 

## Accelerated aging epilepsy

In [ ]:
all_pred_epi_ages = [a.predict(X_e_df) for a in all_models]
all_pred_rns_ages = [a.predict(X_r_df) for a in all_models]

In [ ]:
X_epi_df = pd.concat([X_r_df, X_e_df], axis=0).reset_index(drop=True)

In [ ]:
acc_ag_epi = np.where(y_epilepsy_sub_test < med_s)[0]
acc_ag_epi

In [ ]:
explainer = shap.Explainer(all_models[0])
epi_shap = explainer(X_epi_df)

In [ ]:
all_shap_epi = epi_shap[acc_ag_epi]

In [ ]:
fig = plt.figure(figsize=(20, 8))
gs0 = gridspec.GridSpec(1, 4, figure=fig, width_ratios=[1, 0.05, 1, 0.05])

labels = ['0.03', '0.1', '1', '4', '20']
ticks = np.arange(50)[::10]

shap_values = all_shap_epi
pos_val = np.zeros((len(shap_values), 50, 50)) * np.nan
neg_val = np.zeros((len(shap_values), 50, 50)) * np.nan
pos_extra = np.zeros((len(shap_values), 4)) * np.nan
neg_extra = np.zeros((len(shap_values), 4)) * np.nan
for IDX in range(len(shap_values)):
    _pos = copy.deepcopy(np.reshape(shap_values[IDX].values[:2500], (50, 50)))
    _neg = copy.deepcopy(np.reshape(shap_values[IDX].values[:2500], (50, 50)))
    _pos_e = copy.deepcopy(shap_values[IDX].values[2500:])
    _neg_e = copy.deepcopy(shap_values[IDX].values[2500:])
    
    _pos[_pos < 0] = 0
    pos_val[IDX] = _pos
    _pos_e[_pos_e < 0] = 0
    pos_extra[IDX] = _pos_e
    
    _neg[_neg > 0] = 0
    neg_val[IDX] = _neg
    _neg_e[_neg_e > 0] = 0
    neg_extra[IDX] = _neg_e

    
m_pos = np.nanmean(pos_val, 0)
mask = np.ones_like(m_pos)
mask[m_pos < 0.01] = np.nan
ax0 = fig.add_subplot(gs0[0, 0])
im = ax0.imshow(m_pos * mask, cmap='Reds', vmax=1.2, vmin=0, aspect='auto')
ax0.invert_yaxis()
divider = make_axes_locatable(ax0)
cax = divider.new_vertical(size="5%", pad=.1, pack_start=False)
fig.add_axes(cax)
cb = fig.colorbar(im, cax=cax, orientation="horizontal")
cb.ax.xaxis.set_ticks_position('top')
ax0.set_xticks(ticks)
ax0.set_xticklabels(labels)
ax0.xaxis.set_tick_params(rotation=45)
ax0.set_yticks(ticks)
ax0.set_yticklabels(labels)
ax0.set_ylabel(r'$ITI(k + 1)$ [s]')
ax0.set_xlabel(r'$ITI(k)$ [s]')
ax0.text(-11, 50.1, 'Mean\nShapley\nvalue', size=25)
    
    
ax01 = fig.add_subplot(gs0[0, 1])
ax01.imshow(np.nanmean(pos_extra, 0)[:, None], vmax=1.2, vmin=0, cmap='Reds', aspect='auto')
ax01.yaxis.tick_right()
ax01.set_xticks([])
ax01.set_yticks([0, 1, 2, 3])
ax01.set_yticklabels([new_names[2500], new_names[2501], new_names[2502], new_names[2503]], rotation=90, size=15, verticalalignment='center')

# ax99 = fig.add_subplot(gs0[0, 2])
# ax99.set_xticks([])
# ax99.set_yticks([])
# ax99.spines['right'].set_visible(False)
# ax99.spines['top'].set_visible(False)
# ax99.spines['left'].set_visible(False)
# ax99.spines['bottom'].set_visible(False)

m_neg = np.nanmean(neg_val, 0)
mask = np.ones_like(m_neg)
mask[m_neg > -0.01] = np.nan
ax1 = fig.add_subplot(gs0[0, 2])
im = ax1.imshow(m_neg * mask, cmap="Blues_r", vmin=-.8, vmax=0, aspect='auto')
divider = make_axes_locatable(ax1)
cax = divider.new_vertical(size="5%", pad=.1, pack_start=False)
fig.add_axes(cax)
cb = fig.colorbar(im, cax=cax, orientation="horizontal")
cb.ax.xaxis.set_ticks_position('top')
ax1.invert_yaxis()
ax1.set_xticks(ticks)
ax1.set_xticklabels(labels)
ax1.xaxis.set_tick_params(rotation=45)
ax1.set_yticks(ticks)
ax1.set_yticklabels(labels)
ax1.set_ylabel(r'$ITI(k + 1)$ [s]')
ax1.set_xlabel(r'$ITI(k)$ [s]')
ax1.text(-11, 50.1, 'Mean\nShapley\nvalue', size=25)

ax11 = fig.add_subplot(gs0[0, 3])
ax11.imshow(np.nanmean(neg_extra, 0)[:, None], vmin=-0.8, vmax=0, cmap='Blues_r', aspect='auto')
ax11.yaxis.tick_right()
ax11.set_xticks([])
ax11.set_yticks([0, 1, 2, 3])
ax11.set_yticklabels([new_names[2500], new_names[2501], new_names[2502], new_names[2503]], rotation=90, size=15, verticalalignment='center')

plt.tight_layout()
plt.savefig("figures/xgboost/accelerated_aging_epi.pdf")

In [ ]:
vmin = np.min(np.array([a.values for a in all_shap_epi]))
vmax = np.max(np.array([a.values for a in all_shap_epi]))

shap_values = all_shap_epi
fig = plt.figure(figsize=(15, 34))
main_grid = gridspec.GridSpec(11, 4,  figure=fig, height_ratios=[0.1] + [1] * 10)
cax = fig.add_subplot(main_grid[0, :])

for i, idx in enumerate(acc_ag_epi):
    ax = fig.add_subplot(main_grid[i // 4 + 1, i % 4])
    aa = np.reshape(shap_values[i].values[:2500], (50, 50))
#     aa[np.abs(aa)<1e-3] = 0
    im = ax.imshow(aa, norm=colors.TwoSlopeNorm(vcenter=0, vmin=vmin, vmax=vmax), aspect='auto', cmap='RdBu_r')
    labels = ['0.03', '0.1', '1', '4', '20']
    ticks = np.arange(50)[::10]

#     divider = make_axes_locatable(ax)
#     cax = divider.new_vertical(size="5%", pad=.1, pack_start=False)
#     fig.add_axes(cax)
#     cb = fig.colorbar(im, cax=cax, orientation="horizontal")
#     cb.ax.xaxis.set_ticks_position('top')
    ax.invert_yaxis()
    
    if (i % 4) == 0:
        ax.set_yticks(ticks)
        ax.set_yticklabels(labels)
        ax.set_ylabel(r'$ITI(k + 1)$ [s]')
    else:
        ax.set_yticks([])
    if (i // 4) == 9:
        ax.set_xticks(ticks)
        ax.set_xticklabels(labels)
        ax.xaxis.set_tick_params(rotation=45)
        ax.set_xlabel(r'$ITI(k)$ [s]')
    else:
        ax.set_xticks([])
    
    ax.text(16,42,f"chronological age: { y_epilepsy_sub_test[idx]}\n      predicted age: {int(med_s[idx])}", size=12, alpha=0.85)
    
cb = fig.colorbar(im, cax=cax, orientation="horizontal")
cb.ax.xaxis.set_ticks_position('top')
cax.text(-2, 0, 'Shapley\nvalue', size=25)
plt.tight_layout()
plt.savefig('figures/xgboost/accelerated_aging_full_epi.pdf')

## Accelerated aging stroke

In [ ]:
all_pred_str_ages = [a.predict(X_s_df) for a in all_models]
acc_ag_str = np.where(y_stroke_sub_test < med_s_stroke)[0]
acc_ag_str

In [ ]:
explainer_str = shap.Explainer(all_models[0])
str_shap = explainer_str(X_s_df)

In [ ]:
all_shap_str = str_shap[acc_ag_str]

In [ ]:
fig = plt.figure(figsize=(20, 8))
gs0 = gridspec.GridSpec(1, 4, figure=fig, width_ratios=[1, 0.05, 1, 0.05])

labels = ['0.03', '0.1', '1', '4', '20']
ticks = np.arange(50)[::10]

shap_values = all_shap_str
pos_val = np.zeros((len(shap_values), 50, 50)) * np.nan
neg_val = np.zeros((len(shap_values), 50, 50)) * np.nan
pos_extra = np.zeros((len(shap_values), 4)) * np.nan
neg_extra = np.zeros((len(shap_values), 4)) * np.nan
for IDX in range(len(shap_values)):
    _pos = copy.deepcopy(np.reshape(shap_values[IDX].values[:2500], (50, 50)))
    _neg = copy.deepcopy(np.reshape(shap_values[IDX].values[:2500], (50, 50)))
    _pos_e = copy.deepcopy(shap_values[IDX].values[2500:])
    _neg_e = copy.deepcopy(shap_values[IDX].values[2500:])
    
    _pos[_pos < 0] = 0
    pos_val[IDX] = _pos
    _pos_e[_pos_e < 0] = 0
    pos_extra[IDX] = _pos_e
    
    _neg[_neg > 0] = 0
    neg_val[IDX] = _neg
    _neg_e[_neg_e > 0] = 0
    neg_extra[IDX] = _neg_e

    
m_pos = np.nanmean(pos_val, 0)
mask = np.ones_like(m_pos)
mask[m_pos < 0.01] = np.nan
ax0 = fig.add_subplot(gs0[0, 0])
im = ax0.imshow(m_pos * mask, cmap='Reds', vmax=1.2, vmin=0, aspect='auto')
ax0.invert_yaxis()
divider = make_axes_locatable(ax0)
cax = divider.new_vertical(size="5%", pad=.1, pack_start=False)
fig.add_axes(cax)
cb = fig.colorbar(im, cax=cax, orientation="horizontal")
cb.ax.xaxis.set_ticks_position('top')
ax0.set_xticks(ticks)
ax0.set_xticklabels(labels)
ax0.xaxis.set_tick_params(rotation=45)
ax0.set_yticks(ticks)
ax0.set_yticklabels(labels)
ax0.set_ylabel(r'$ITI(k + 1)$ [s]')
ax0.set_xlabel(r'$ITI(k)$ [s]')
ax0.text(-11, 50.1, 'Mean\nShapley\nvalue', size=25)
    
    
ax01 = fig.add_subplot(gs0[0, 1])
ax01.imshow(np.nanmean(pos_extra, 0)[:, None], vmax=1.2, vmin=0, cmap='Reds', aspect='auto')
ax01.yaxis.tick_right()
ax01.set_xticks([])
ax01.set_yticks([0, 1, 2, 3])
ax01.set_yticklabels([new_names[2500], new_names[2501], new_names[2502], new_names[2503]], rotation=90,size=15, verticalalignment='center')

# ax99 = fig.add_subplot(gs0[0, 2])
# ax99.set_xticks([])
# ax99.set_yticks([])
# ax99.spines['right'].set_visible(False)
# ax99.spines['top'].set_visible(False)
# ax99.spines['left'].set_visible(False)
# ax99.spines['bottom'].set_visible(False)

m_neg = np.nanmean(neg_val, 0)
mask = np.ones_like(m_neg)
mask[m_neg > -0.01] = np.nan
ax1 = fig.add_subplot(gs0[0, 2])
im = ax1.imshow(m_neg * mask, cmap="Blues_r", vmin=-0.8, vmax=0, aspect='auto')
divider = make_axes_locatable(ax1)
cax = divider.new_vertical(size="5%", pad=.1, pack_start=False)
fig.add_axes(cax)
cb = fig.colorbar(im, cax=cax, orientation="horizontal")
cb.ax.xaxis.set_ticks_position('top')
ax1.invert_yaxis()
ax1.set_xticks(ticks)
ax1.set_xticklabels(labels)
ax1.xaxis.set_tick_params(rotation=45)
ax1.set_yticks(ticks)
ax1.set_yticklabels(labels)
ax1.set_ylabel(r'$ITI(k + 1)$ [s]')
ax1.set_xlabel(r'$ITI(k)$ [s]')
ax1.text(-11, 50.1, 'Mean\nShapley\nvalue', size=25)

ax11 = fig.add_subplot(gs0[0, 3])
ax11.imshow(np.nanmean(neg_extra, 0)[:, None], vmin=-0.8, vmax=0, cmap='Blues_r', aspect='auto')
ax11.yaxis.tick_right()
ax11.set_xticks([])
ax11.set_yticks([0, 1, 2, 3])
ax11.set_yticklabels([new_names[2500], new_names[2501], new_names[2502], new_names[2503]], rotation=90, size=15, verticalalignment='center')

plt.tight_layout()
plt.savefig("figures/xgboost/accelerated_aging_stroke.pdf")

In [ ]:
acc_ag_str.shape

In [ ]:
vmin = np.min(np.array([a.values for a in all_shap_str]))
vmax = np.max(np.array([a.values for a in all_shap_str]))

shap_values = all_shap_str
fig = plt.figure(figsize=(15, 25))

main_grid = gridspec.GridSpec(8, 4,  figure=fig, height_ratios=[0.1] + [1] * 7)
cax = fig.add_subplot(main_grid[0, :])

for i, idx in enumerate(acc_ag_str[:28]):
    ax = fig.add_subplot(main_grid[i // 4 + 1, i % 4])
    aa = np.reshape(shap_values[i].values[:2500], (50, 50))
#     aa[np.abs(aa)<1e-3] = 0
    im = ax.imshow(aa, norm=colors.TwoSlopeNorm(vcenter=0, vmin=vmin, vmax=vmax), aspect='auto', cmap='RdBu_r')
    labels = ['0.03', '0.1', '1', '4', '20']
    ticks = np.arange(50)[::10]

#     divider = make_axes_locatable(ax)
#     cax = divider.new_vertical(size="5%", pad=.1, pack_start=False)
#     fig.add_axes(cax)
#     cb = fig.colorbar(im, cax=cax, orientation="horizontal")
#     cb.ax.xaxis.set_ticks_position('top')
    ax.invert_yaxis()
    
    if (i % 4) == 0:
        ax.set_yticks(ticks)
        ax.set_yticklabels(labels)
        ax.set_ylabel(r'$ITI(k + 1)$ [s]')
    else:
        ax.set_yticks([])
        
    if (i // 4) == 6:
        ax.set_xticks(ticks)
        ax.set_xticklabels(labels)
        ax.xaxis.set_tick_params(rotation=45)
        ax.set_xlabel(r'$ITI(k)$ [s]')
    else:
        ax.set_xticks([])
    
    ax.text(16,42,f"chronological age: { y_stroke_sub_test[idx]}\n      predicted age: {int(med_s_stroke[idx])}", size=12, alpha=0.85)

cb = fig.colorbar(im, cax=cax, orientation="horizontal")
cb.ax.xaxis.set_ticks_position('top')

plt.tight_layout()
plt.savefig('figures/xgboost/accelerated_aging_full_stroke.pdf')

# Supplementary 1

In [ ]:
more_info = pd.read_csv('more_info_for_review_ml_age.csv')
more_info = more_info.drop(503)
more_info.insert(1, "entropy", e_708)
more_info = more_info[more_info.age > 0]
more_info = more_info[more_info.n_taps > 100]
more_info = more_info[more_info.n_days > 7]
more_info.head()

In [ ]:
# need to put NaN where info is missing
more_info.phoneModel.replace(0, np.nan, inplace=True)
more_info.n_apps.replace([0, 1], np.nan, inplace=True)
more_info['fingerdness(1)'].replace(-1, np.nan, inplace=True)
more_info['fingerdness(2)'].replace(-1, np.nan, inplace=True)
more_info.yearsused.replace(-1, np.nan, inplace=True)
more_info.mcs.replace(-1, np.nan, inplace=True)
more_info.pcs.replace(-1, np.nan, inplace=True)
more_info.tail(20)

In [ ]:
info_m = more_info[more_info.gender == 1]
info_f = more_info[more_info.gender == 2]

info_m = info_m.sort_values('age')
info_f = info_f.sort_values('age')
len(info_m), len(info_f)
info_m.head()

In [ ]:
len(info_m), len(info_f)

In [ ]:
fig = plt.figure(figsize=(15, 20))
gs0 = gridspec.GridSpec(2, 6, figure=fig, height_ratios=[len(info_m) / len(info_f), 1], wspace=0.5)

# male
cmap = "Blues"
ax = [fig.add_subplot(gs0[0, i]) for i in range(6)]
im0 = ax[0].imshow(np.log10(info_m['usage']).values[:, None], aspect='auto', interpolation=None, cmap=cmap)
im1 = ax[1].imshow(info_m['phoneModel'].values[:, None], aspect='auto', interpolation=None, cmap=cmap)
im2 = ax[2].imshow(info_m['entropy'].values[:, None], aspect='auto', interpolation=None, cmap=cmap)
im4 = ax[4].imshow(info_m['mcs'].values[:, None], aspect='auto', interpolation=None, cmap=cmap)
im5 = ax[5].imshow(info_m['pcs'].values[:, None], aspect='auto', interpolation=None, cmap=cmap)
for _ax, _im, _cb_r, _title in zip([ax[0], ax[1], ax[2], ax[4], ax[5]], 
                    [im0, im1, im2, im4, im5], 
                    [[2, 4], [5, 10], [6, 9], [0, 85], [0, 85]], 
                    ["Usage \n(Log10 no. of\n interactions/day)", "Screen size \n(inches)", "Entropy (bits)", "MCS (SF36)", "PCS (SF36)"]):
    _ax.grid(0)
    _ax.set_xticks([])
    _ax.set_yticks([])
    divider = make_axes_locatable(_ax)
    cax = divider.new_vertical(size="5%", pad=.1, pack_start=False)
    fig.add_axes(cax)
    cb = fig.colorbar(_im, cax=cax, orientation="horizontal")
    cb.ax.xaxis.set_ticks_position('top')
#     cb.ax.xaxis.set_ticks(_cb_r)
    _ax.set_title(_title, pad=70, size=12)

ax[0].set_yticks(range(0, len(info_m), 30))
ax[0].set_yticklabels([info_m.age.values[i] for i in range(0, len(info_m), 30)])
ax[0].set_ylabel("Age \u2642 [years]")

    
for _ax in [ax[3]]:
    _ax.grid(0)
    _ax.set_xticks([])
    _ax.set_yticks([])
    _ax.spines['right'].set_visible(False)
    _ax.spines['left'].set_visible(False)
    _ax.spines['top'].set_visible(False)
    _ax.spines['bottom'].set_visible(False)

# female
ax = [fig.add_subplot(gs0[1, i]) for i in range(6)]
cmap = "Reds"
im0 = ax[0].imshow(np.log10(info_f['usage']).values[:, None], aspect='auto', interpolation=None, cmap=cmap)
im1 = ax[1].imshow(info_f['phoneModel'].values[:, None], aspect='auto', interpolation=None, cmap=cmap)
im2 = ax[2].imshow(info_f['entropy'].values[:, None], aspect='auto', interpolation=None, cmap=cmap)
im4 = ax[4].imshow(info_f['mcs'].values[:, None], aspect='auto', interpolation=None, cmap=cmap)
im5 = ax[5].imshow(info_f['pcs'].values[:, None], aspect='auto', interpolation=None, cmap=cmap)
for _ax, _im, _cb_r in zip([ax[0], ax[1], ax[2], ax[4], ax[5]], 
                    [im0, im1, im2, im4, im5],
                    [[2, 4], [5, 10], [6, 9], [0, 85], [0, 85]]):
    _ax.grid(0)
    _ax.set_xticks([])
    _ax.set_yticks([])
    divider = make_axes_locatable(_ax)
    cax = divider.new_vertical(size="5%", pad=.1, pack_start=False)
    fig.add_axes(cax)
    cb = fig.colorbar(_im, cax=cax, orientation="horizontal")
    cb.ax.xaxis.set_ticks_position('top')
#     cb.ax.xaxis.set_ticks(_cb_r)
    
for _ax in [ax[3]]:
    _ax.grid(0)
    _ax.set_xticks([])
    _ax.set_yticks([])
    _ax.spines['right'].set_visible(False)
    _ax.spines['left'].set_visible(False)
    _ax.spines['top'].set_visible(False)
    _ax.spines['bottom'].set_visible(False)

ax[0].set_yticks(range(0, len(info_f), 30))
ax[0].set_yticklabels([info_f.age.values[i] for i in range(0, len(info_f), 30)])
ax[0].set_ylabel("Age \u2640 [years]")
plt.tight_layout()
plt.savefig('figures/xgboost/figure_1_more_info_columns.pdf')